In [ ]:
import pandas as pd
import csv

In [ ]:
#MERGE query data to event data

In [ ]:
df_panama_papers = pd.read_table('data/final/query_panama_papers_data.txt', sep='\t', encoding='utf-8', header=0)
df_offshore = pd.read_table('data/final/query_offshore_data.txt', sep='\t', encoding='utf-8', header=0)
df_leak = pd.read_table('data/final/query_leak_data.txt', sep='\t', encoding='utf-8', header=0)
df_fonseca = pd.read_table('data/final/query_fonseca_data.txt', sep='\t', encoding='utf-8', header=0)
df_tax_haven = pd.read_table('data/final/query_tax_haven_data.txt', sep='\t', encoding='utf-8', header=0)

In [ ]:
df_panama_papers['query'] = 'panama papers'
df_offshore['query'] = 'offshore'
df_leak['query'] = 'leak'
df_fonseca['query'] = 'fonseca'
df_tax_haven['query'] = 'tax haven'

In [ ]:
df = pd.concat([df_panama_papers, df_offshore, df_leak, df_fonseca, df_tax_haven])
df.shape

In [ ]:
#handle event overlap between queries
#not query specific
df = df.drop_duplicates('id_str')
df.shape

In [ ]:
#convert date format to python datetime
df['created_at'] = pd.to_datetime(df['created_at'])

In [ ]:
#sort data on date posted ascending
df = df.sort_values(by='created_at', ascending=True)

In [ ]:
#slice data on event dates
df = df[(df['created_at'] >= '2016-04-02 00:00:00') & (df['created_at'] <= '2016-04-15 00:00:00')]

In [ ]:
#MASTER FILE

In [ ]:
df_master = df[['id_str', 'created_at', 'text', 'is_retweet', 'tweet_type']]

In [ ]:
df_master['event'] = 'Panama Papers'
df_master['event_description'] = 'A leak of millions of documents detailing global tax evasion named “The Panama Papers”'

In [ ]:
df_master = df_master.reset_index()
df_master.columns = ['master_id', 'twitter_id', 'created_at', 'text', 'is_retweet', 'tweet_type', 'event', 'event_description']

In [ ]:
#save id, text as master, with index
df_master.to_csv('data/final/event_panama_papers_master.txt', sep='\t', encoding='utf-8', header=True, index=False)

In [ ]:
#save sample for annotation
#no retweets, only text
#sample 1K
smpl = df_master[(df_master['is_retweet'] == False) & (df_master['tweet_type'] == 'text')].sample(1000)
#format for crowdflower import
#delimited by ;
#double quote strings
smpl.to_csv('data/final/event_panama_papers_master_sample.csv', sep=',', quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8', header=True, index=False)